## 문장 임베딩 활용

In [3]:
# 문장 유사도 계산
from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")


In [5]:
words = ['학교', '공부', '게임', '자전거']

dense_embedded = model_sentence.encode(words)
dense_embedded
len(dense_embedded[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

768

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(dense_embedded)


array([[0.9999999 , 0.59507424, 0.30069774, 0.20964187],
       [0.59507424, 0.9999998 , 0.41403902, 0.16175744],
       [0.30069774, 0.41403902, 1.0000001 , 0.29132563],
       [0.20964187, 0.16175744, 0.29132563, 1.0000001 ]], dtype=float32)

In [40]:
sentences = [
    "그 식당은 파리를 날린다",
    "그 식당에는 손님이 없다",
    # "그 식당에서는 드론을 날린다",
    # "파리가 식당에 날아다닌다",
    # "학생이 자전거타고 학교에 간다",
    "오늘 날씨가 정말 좋네요.",
    "오늘 날씨가 맑고 기분이 좋아요.",
    "인공지능을 활용하면 업무 효율을 높일 수 있다.",
    "AI 기술을 적용하면 생산성이 향상된다.",
    "강아지는 사람에게 충성심이 강한 동물이다.",
    "주식시장은 변동성이 크고 예측하기 어렵다."
]
# sentences = [
#     "우리 팀은 이번 시즌 5연패를 기록하며 순위가 하락했다.",  # (1) 연속 패배
#     "그 선수는 연패를 끊기 위해 전략을 변경했다.",  # (1) 연속 패배
#     "챔피언십 경기에서 연패를 당한 팀은 다음 시즌을 대비하고 있다.",  # (1) 연속 패배
#     "이 건물의 기둥은 정교한 연패 문양으로 장식되어 있다.",  # (2) 연속적인 무늬
#     "고대 유적에서는 독특한 연패 형식의 조각을 볼 수 있다.",  # (2) 연속적인 무늬
#     "그 도자기의 가장자리는 연패 무늬로 아름답게 꾸며져 있었다.",  # (2) 연속적인 무늬
#     "그 선수는 대회에서 3년 연속 우승하며 연패의 위업을 달성했다.",  # (3) 연속 재패
#     "우리 팀은 세계 챔피언십에서 2연패를 기록하며 최강의 자리를 유지했다."  # (3) 연속 재패
# ]
sentence_embeddings = model_sentence.encode(sentences)

similarities = model_sentence.similarity(sentence_embeddings, sentence_embeddings)
# similarities = model_sentence.similarity([sentence_embeddings[-4]], sentence_embeddings)
similarities

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[ 1.0000,  0.6306, -0.0144, -0.0135,  0.0767,  0.1518,  0.1863,  0.1206],
        [ 0.6306,  1.0000,  0.0049,  0.0031,  0.0356,  0.1423,  0.1492,  0.1533],
        [-0.0144,  0.0049,  1.0000,  0.9317,  0.1768,  0.1984,  0.1988,  0.1081],
        [-0.0135,  0.0031,  0.9317,  1.0000,  0.1743,  0.1640,  0.1692,  0.1034],
        [ 0.0767,  0.0356,  0.1768,  0.1743,  1.0000,  0.8091,  0.1696,  0.2261],
        [ 0.1518,  0.1423,  0.1984,  0.1640,  0.8091,  1.0000,  0.2135,  0.1819],
        [ 0.1863,  0.1492,  0.1988,  0.1692,  0.1696,  0.2135,  1.0000,  0.2197],
        [ 0.1206,  0.1533,  0.1081,  0.1034,  0.2261,  0.1819,  0.2197,  1.0000]])

## 기존 라이브러리 활용

In [34]:
from sentence_transformers import models

word_embedding_layers = models.Transformer('klue/roberta-base')
word_embedding_layers # 기존 학습된 모델에서 임베딩 층만(transformer layer)만 가져옴
word_embedding_layers.get_word_embedding_dimension()

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


768

In [35]:
# output 구성
pooling_layer = models.Pooling(word_embedding_layers.get_word_embedding_dimension())

In [37]:
custom_sentence_model = SentenceTransformer(modules=[word_embedding_layers, pooling_layer]) # 
custom_sentence_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [43]:
## 활용 
custom_sentence_embeddings = custom_sentence_model.encode(sentences)

similarities = custom_sentence_model.similarity(custom_sentence_embeddings, custom_sentence_embeddings)
# similarities = model_sentence.similarity([sentence_embeddings[-4]], sentence_embeddings)
similarities

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[1.0000, 0.8397, 0.7226, 0.6604, 0.6811, 0.7108, 0.7044, 0.6436],
        [0.8397, 1.0000, 0.7199, 0.6881, 0.6921, 0.7127, 0.7380, 0.6835],
        [0.7226, 0.7199, 1.0000, 0.9347, 0.7160, 0.7274, 0.7303, 0.7175],
        [0.6604, 0.6881, 0.9347, 1.0000, 0.6894, 0.6999, 0.7078, 0.7067],
        [0.6811, 0.6921, 0.7160, 0.6894, 1.0000, 0.9312, 0.7470, 0.7737],
        [0.7108, 0.7127, 0.7274, 0.6999, 0.9312, 1.0000, 0.7709, 0.7775],
        [0.7044, 0.7380, 0.7303, 0.7078, 0.7470, 0.7709, 1.0000, 0.7603],
        [0.6436, 0.6835, 0.7175, 0.7067, 0.7737, 0.7775, 0.7603, 1.0000]])

In [42]:
len(custom_sentence_embeddings[0])

768

## 이미지 유사도 

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/clip-ViT-B-32")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [4, 4]